In [3]:
import pandas as pd
from skmultiflow.trees import HoeffdingTree
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
fake_news = "D:/Desktop/Fake_News_Dataset/ISOT_fake.csv"
true_news = "D:/Desktop/Fake_News_Dataset/ISOT_true.csv"
true_df = pd.read_csv(fake_news)
fake_df = pd.read_csv(true_news)

In [5]:
true_df["label"] = 1  # 1 for true news
fake_df["label"] = 0  # 0 for fake news

In [6]:
# Combine datasets
combined_df = pd.concat([true_df, fake_df], ignore_index=True)

# Shuffle data
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [7]:
# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
vectorizer.fit(combined_df['text'])
# Initialize Hoeffding Tree classifier
ht = HoeffdingTree()

c:\Users\Kartik Gounder\Desktop\Projects\.venv\Lib\site-packages\skmultiflow\trees\hoeffding_tree.py:32: FutureWarning: 'HoeffdingTree' has been renamed to 'HoeffdingTreeClassifier' in v0.5.0.
The old name will be removed in v0.7.0
  warnings.warn("'HoeffdingTree' has been renamed to 'HoeffdingTreeClassifier' in v0.5.0.\n"


In [8]:
# Incremental training
for i in range(len(combined_df)):
    text = combined_df.loc[i, 'text']
    label = combined_df.loc[i, 'label']
    
    # Vectorize the text
    X_partial = vectorizer.transform([text]).toarray()[0]
    
    # Train the Hoeffding Tree incrementally
    ht.partial_fit([X_partial], [int(label)])  # Convert label to integer

c:\Users\Kartik Gounder\Desktop\Projects\.venv\Lib\site-packages\skmultiflow\bayes\utils.py:39: RuntimeWarning: overflow encountered in scalar multiply
  tmp = votes[class_index] * obs.probability_of_attribute_value_given_class(
c:\Users\Kartik Gounder\Desktop\Projects\.venv\Lib\site-packages\skmultiflow\bayes\utils.py:39: RuntimeWarning: invalid value encountered in scalar multiply
  tmp = votes[class_index] * obs.probability_of_attribute_value_given_class(


In [12]:
y_true = combined_df['label']
X_test = combined_df['text']

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(combined_df['text'], combined_df['label'], test_size = 0.2, random_state = 0)

In [14]:
# Convert sparse matrix to dense array
X_test_dense = X_test.toarray()

# Predict using the Hoeffding Tree classifier
y_pred_incremental = ht.predict(X_test_dense)

# Calculate accuracy
incremental_accuracy = accuracy_score(y_true, y_pred_incremental)
print("Incremental model accuracy:", incremental_accuracy)


Incremental model accuracy: 0.9839859236491603


In [20]:
import pickle

# Get the model's state dictionary
model_state = ht.get_model_state()

# Save the model's state to a file
with open('hoeffding_tree_model.pkl', 'wb') as file:
    pickle.dump(model_state, file)

# Load the model's state from the file
with open('hoeffding_tree_model.pkl', 'rb') as file:
    loaded_model_state = pickle.load(file)

# Reconstruct the model
loaded_ht = HoeffdingTree()
loaded_ht.set_model_state(loaded_model_state)


AttributeError: 'HoeffdingTreeClassifier' object has no attribute 'get_model_state'

In [18]:
dump(ht, 'hoeffding_tree_model.joblib')

PicklingError: Can't pickle <class 'skmultiflow.trees.hoeffding_tree.HoeffdingTreeClassifier'>: it's not found as skmultiflow.trees.hoeffding_tree.HoeffdingTreeClassifier